In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os
import shutil
from sklearn.model_selection import train_test_split
original_dataset_dir = 'D:/tester01'
classes = ['normal', 'cataract', 'surgery']
base_dir = 'D:/splitted_tester01'

if not os.path.exists(base_dir):
    os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes:
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))
# 设置训练、验证和测试集的比例
train_size = 0.7
validation_size = 0.2
test_size = 0.1

for cls in classes:
    # 获取类别目录下的所有文件
    files = os.listdir(os.path.join(original_dataset_dir, cls))

    # 分割数据集
    train_files, test_files = train_test_split(files, test_size=test_size, random_state=42)
    train_files, val_files = train_test_split(train_files, test_size=validation_size/(train_size+validation_size), random_state=42)

    # 复制文件到新的训练、验证和测试目录
    for file in train_files:
        src = os.path.join(original_dataset_dir, cls, file)
        dst = os.path.join(train_dir, cls, file)
        shutil.copyfile(src, dst)

    for file in val_files:
        src = os.path.join(original_dataset_dir, cls, file)
        dst = os.path.join(validation_dir, cls, file)
        shutil.copyfile(src, dst)

    for file in test_files:
        src = os.path.join(original_dataset_dir, cls, file)
        dst = os.path.join(test_dir, cls, file)
        shutil.copyfile(src, dst)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'D:/splitted_tester01/train'  
validation_dir = 'D:/splitted_tester01/validation'  
test_dir = 'D:/splitted_tester01/test' 

# 训练数据生成器
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# 验证数据生成器
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# 测试数据生成器
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 2142 images belonging to 3 classes.
Found 614 images belonging to 3 classes.
Found 308 images belonging to 3 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(
    train_generator,
    epochs=10,  
    validation_data=validation_generator,
    verbose=1)

Epoch 1/10
67/67 [==============================] - 37s 543ms/step - loss: 0.6242 - accuracy: 0.7624 - val_loss: 0.2812 - val_accuracy: 0.9039
Epoch 2/10
67/67 [==============================] - 34s 508ms/step - loss: 0.2040 - accuracy: 0.9342 - val_loss: 0.1993 - val_accuracy: 0.9267
Epoch 3/10
67/67 [==============================] - 35s 514ms/step - loss: 0.1595 - accuracy: 0.9458 - val_loss: 0.2341 - val_accuracy: 0.9365
Epoch 4/10
67/67 [==============================] - 36s 534ms/step - loss: 0.1095 - accuracy: 0.9627 - val_loss: 0.1927 - val_accuracy: 0.9365
Epoch 5/10
67/67 [==============================] - 36s 540ms/step - loss: 0.0930 - accuracy: 0.9678 - val_loss: 0.1625 - val_accuracy: 0.9528
Epoch 6/10
67/67 [==============================] - 39s 582ms/step - loss: 0.0615 - accuracy: 0.9757 - val_loss: 0.2529 - val_accuracy: 0.9251
Epoch 7/10
67/67 [==============================] - 40s 594ms/step - loss: 0.0563 - accuracy: 0.9832 - val_loss: 0.1618 - val_accuracy: 0.9446

In [13]:
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test accuracy:', test_accuracy)

10/10 [==============================] - 2s 243ms/step - loss: 0.1434 - accuracy: 0.9643
Test accuracy: 0.9642857313156128


In [14]:
model.save('TY model.h5')